In [100]:
from flask import Flask, render_template, request, jsonify
from flaskext.mysql import MySQL

import pymysql  # mysql error handling
import itertools
import json

In [95]:
app = Flask(__name__)
app.config.from_pyfile('config.py')

mysql = MySQL()
mysql.init_app(app)

conn = mysql.connect()
cursor = conn.cursor()

## Registro

In [123]:
def register(socio):
    '''
    Args: 
        Socio es un diccionario con atributos 
        ('apPaterno', 'apMaterno', 'nombre', 'edad', 'genero', 'email', 'passwd')
    
    Return:
        Diccionario donde el valor para la clave 'email_used' es True si ya existe un email por ingresar, 
        valor para success True en caso de hacer el registro correctamente.
    '''
    res = {'email_used': False, 'success': False}
    try:
        args_order = ('apPaterno', 'apMaterno', 'nombre', 'edad', 'genero', 'email', 'passwd')
        args = tuple(socio[arg] for arg in args_order)
        cursor.callproc('insert_socio', args)
        res['success'] = True
    except pymysql.err.IntegrityError as err:
        if 'c_uniq_email_passwd' in str(err):
            res['email_used'] = True
        print(err)
    finally:
        conn.commit()    
        return res

In [125]:
socio = {"apPaterno": "Mendoza", "apMaterno": "Hernandez", "nombre": "Alma", 
        "edad": 22, "genero": "F", "email": "almis@gmail.com", "passwd": "frio"}
register(socio)

{'email_used': False, 'success': True}

In [126]:
register(socio)

(1062, "Duplicate entry 'almis@gmail.com' for key 'socio.c_uniq_email_passwd'")


{'email_used': True, 'success': False}

In [127]:
socio = {"apPaterno": "Fragoso", "apMaterno": "Hernandez", "nombre": "Abigail", 
        "edad": 19, "genero": "F", "email": "almis@gmail.com", "passwd": "frio"}
register(socio)

(1062, "Duplicate entry 'almis@gmail.com' for key 'socio.c_uniq_email_passwd'")


{'email_used': True, 'success': False}

## Login

In [85]:
def login(email, passwd):
    cursor.execute('select passwd, idSocio from socio where email = %s ', email)
    passwd_id_from_db = cursor.fetchone()
    res = {'email_found': False, 'correct_passwd': False, 'idSocio': False}
    if passwd_id_from_db:
        res['email_found'] = True
        res['correct_passwd'] = passwd == passwd_id_from_db[0]
        if res['correct_passwd']:
            res['idSocio'] = passwd_id_from_db[1]
    return res

In [128]:
email, passwd = 'almis@gmail.com', 'frio'
login(email, passwd)

{'email_found': True,
 'correct_passwd': True,
 'idSocio': '1f3fd27234c16bd01f4c'}

In [129]:
passwd = 'incorrecto'
login(email, passwd)

{'email_found': True, 'correct_passwd': False, 'idSocio': False}

In [130]:
email = 'noexiste@gmail.com'
login(email, 'falsooo')

{'email_found': False, 'correct_passwd': False, 'idSocio': False}